# Тестовое задание
## Парсинг диалогов
Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [25]:
import pandas as pd

In [27]:
# данные
data = pd.read_csv('test_data.csv')
data.sample(5)

,dlg_id,line_n,role,text
433,5,96,manager,Хэлп которая
308,4,6,manager,Я понял ну мы хотели бы просто предложить тако...
121,1,12,manager,Угу
59,0,59,client,По какой причине я должен принять решение в ва...
25,0,25,client,+ месяц 2 3 все что угодно


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


Датафрейм содержит 480 строк и 4 столбца:
- `dlg_id` - номер диалога (0-5)
- `line_n` - номер сообщения (реплики) в диалоге
- `role` - менеджер/клиент
- `text` - текст сообщения 

Каждая реплика состоит из одного предложения (без знаков препинания).

In [29]:
# пример сообщения
data.iloc[167]['text']

'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там'

### 1. Подготовка к решению задачи

Необходимо выделить следующие типы именованных сущностей:
- `PER` - имя менеджера
- `ORG` - название компании

Поскольку данных недостаточно для обучения собственной модели, были предприняты попытки использования уже предобученных моделей, имеющих поддержку русского языка (из библиотек *Stanza*, *PullEnti*, *Natasha*, *Spacy*). К сожалению, ни одна из выбранных моделей не показала удовлетворительного результата (ниже пример кода для Spacy). Возможно, это связано с тем, что модели обучались на письменных текстах (новости, медиа), где имена собственные начинаются с прописных букв.

In [30]:
# !pip install spacy
# !python -m spacy download ru_core_news_md
import spacy

nlp = spacy.load("ru_core_news_md")
doc = nlp(data.iloc[167]['text'])    
print(doc.ents)

()


In [31]:
russian_names = pd.read_csv('../bewise_test/russian_names.csv', sep=';')
russian_names.head()

,ID,Name,Sex,PeoplesCount,WhenPeoplesCount,Source
0,19903,Аалия,Ж,13,23.06.2016 13:39:41,myData.biz
1,19904,Аанжелла,Ж,0,23.06.2016 13:39:46,myData.biz
2,19905,Аба,Ж,1000,23.06.2016 13:39:55,myData.biz
3,19906,Абав,Ж,0,23.06.2016 13:40:02,myData.biz
4,19907,Абам,Ж,32,23.06.2016 13:40:11,myData.biz


In [33]:
# список русских имен
russian_names_list = russian_names['Name'].unique()

### 2. Вспомогательные функции

Решение на основе nltk и pymorphy2. Для отнесения выбранного слова к имени дополнительно был загружен [список русских имен](https://mydata.biz/ru/catalog/databases/names_db) (см. выше)

In [154]:
# !pip install nltk
# !pip install pymorphy2
# nltk.download('punkt')

In [155]:
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Функция **is_self_presentation** находит в сообщении паттерн представления менеджера

In [156]:
def is_self_presentation(message, name):
    if (
        f'меня {name} зовут' in message or
        f'меня зовут {name}' in message or
        f'это {name}' in message
    ):
        return True
    return False

In [41]:
# проверка работоспособности функции
print(is_self_presentation('меня зовут ангелина', 'ангелина'))
text = data.iloc[167]['text']
print(is_self_presentation(text.lower(), 'ангелина'))

True
True


Функция **names_recognize** находит имя человека в сообщении и возвращает его, если это имя - имя менеджера. Названия компаний игнорирует.

In [157]:
def names_recognize(message):
    global morph
    prob_thresh = 0.2  # порог для определения слова-имени
    name_list = []     # на случай, если в сообщении не одно имя
    
    for word in nltk.word_tokenize(message.lower()):
        is_name = any(('Name' in p.tag and p.score >= prob_thresh) for p in morph.parse(word))
        if is_name:
            name_list.append(word)
            
    for name in name_list:
        if is_self_presentation(message.lower(), name):
            return name

In [49]:
# проверка работоспособности функции
print(names_recognize(data.iloc[3]['text']))
print(names_recognize(data.iloc[1]['text']))
print(names_recognize(data.iloc[251]['text']))

ангелина
None
максим


Функция **company_recognize** находит название компании в сообщении. В том числе учитываютс случаи, когда название компании состоит из двух и более слов.

In [158]:
def company_recognize(message):
    global morph
    if 'компания' not in message.lower():
        return
    result = ''
    for word in nltk.word_tokenize(message.lower()):
        # является ли слово началом названия компании
        is_company = any(f'компания {word}' in message.lower() and 'NOUN' in p.tag for p in morph.parse(word))
        # является ли слово частью названия компании
        is_part_name = any(
            ((len(result) > 0) and f'{result}{word}' in message.lower()) and 
            'NOUN' in p.tag for p in morph.parse(word)
        )
        
        if is_company or is_part_name:
            result += word + ' '
            
    return result

In [153]:
# проверка работоспособности функции
print(company_recognize(data.iloc[3]['text']))
print(company_recognize(data.iloc[1]['text']))
print(company_recognize(data.iloc[251]['text']))

диджитал бизнес 
None
китобизнес 


Функции **is_greeting** и **is_goodbye_message** определяют, является ли сообщение приветствием и прощанием соответственно.

In [159]:
def is_greeting(message):
    if (
        'добры' in message.lower() or         # добрый день, добрый вечер...
        'здравствуй' in message.lower() or    # здравствуй, здравствуйте
        'привет'in message.lower()            # привет, приветствую
    ):          
        return True

    return False

In [160]:
def is_goodbye_message(message):
    if (
        'свидани' in message.lower() or   # до свидания (и если вдруг ошибка в написании слова)
        'встречи' in message.lower() or   # до встречи 
        'хороше' in message.lower() or    # хорошего дня, хорошего понедельника... НО: не 'хорошо' - это не приветствие
        'добро' in message.lower()        # всего доброго, доброго дня...
    ): 
        return True
    
    return False

Функция **check_manager** определяет, выполнил ли менеджер требование (поздороваться в начале диалога и попрощаться в конце)

In [161]:
def check_manager(greeting, goodbye):
    if greeting and goodbye:
        return True
    return False

### 3. Парсинг текста

#### Составим парсер таким образом: 
- **вход**: подаются сообщения диалога в виде таблицы (датафрейма)
- **выход**: печатается
    - имя менеджера
    - имя компании
    - номер и текст реплики приветствия
    - номер и текст реплики само-представления менеджера
    - номер и текст реплики прощания
    - соблюдено ли требование

In [198]:
def parser_dialog(dialog):
    dialog_info = {}
    dtable = dialog.copy()
    dtable['names'] = dtable['text'].apply(names_recognize)
    dtable['greeting'] = dtable['text'].apply(is_greeting)
    dtable['goodbye'] = dtable['text'].apply(is_goodbye_message)
    
    # уникальные имена
    dialog_names = set((';'.join(list(dtable['names'].unique()))).split(';'))
    print(dialog_names)
    
    # извлекаем имя менеджера и компании
    dialog_info = extract_names(dialog_names, dialog_info)
    if 'manager_name' not in dialog_info:
        dialog_info['manager_name'] = 'unknown'
    if 'company_name' not in dialog_info:
        dialog_info['company_name'] = 'unknown'
    
    dtable['self_present'] = dtable.apply(lambda x: is_self_presentation(x['text'].lower(), dialog_info['manager_name'].lower()), axis=1)
    print(dtable.query('self_present')['self_present'])
    
    # извлекаем приветственное сообщение
    dgreeting = dtable.query('greeting')
    dialog_info['greeting_message'] = {row['line_n']: row['text'] for index, row in dgreeting.iterrows()}
    
    # извлекаем прощальное сообщение
    dgoodbye = dtable.query('goodbye')
    dialog_info['goodbye_message'] = {row['line_n']: row['text'] for index, row in dgoodbye.iterrows()}
    
    # извлекаем реплику, где менеджер представляется
    dself = dtable.query('self_present')
    dialog_info['manager_self_presentation'] = {row['line_n']: row['text'] for index, row in dself.iterrows()}
    
    # выполнил ли менеджер поставленное условие (поздороваться и попрощаться)
    dialog_info['requirement_done'] = (dgreeting.shape[0] != 0 and dgoodbye.shape[0] != 0)
    
    return dialog_info

In [199]:
dialogs = {}
for i in data['dlg_id'].unique():
    dialogs[i] = parser_dialog(data.query('dlg_id == @i and role == "manager"'))

{'', 'company=диджитал бизнес', 'manager=ангелина'}
3    True
Name: self_present, dtype: bool
{'', 'company=диджитал бизнес', 'manager=ангелина'}
111    True
Name: self_present, dtype: bool
{'', 'company=диджитал бизнес', 'manager=ангелина'}
167    True
Name: self_present, dtype: bool
{'', 'company=китобизнес', 'manager=максим'}
251    True
Name: self_present, dtype: bool
{''}
Series([], Name: self_present, dtype: bool)
{'', 'manager=анастасия'}
338    True
Name: self_present, dtype: bool


In [200]:
print(dialogs)

{0: {'company_name': 'Диджитал бизнес', 'manager_name': 'Ангелина', 'greeting_message': {1: 'Алло здравствуйте'}, 'goodbye_message': {108: 'Всего хорошего до свидания'}, 'manager_self_presentation': {3: 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'}, 'requirement_done': True}, 1: {'company_name': 'Диджитал бизнес', 'manager_name': 'Ангелина', 'greeting_message': {1: 'Алло здравствуйте'}, 'goodbye_message': {53: 'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего', 54: 'До свидания'}, 'manager_self_presentation': {2: 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается'}, 'requirement_done': True}, 2: {'company_name': 'Диджитал бизнес', 'manager_name': 'Ангелина', 'greeting_message': {2: 'Алло здравствуйте'}, 'goodbye_message': {}, 'manager_self_presentation': {3: 'Меня зовут ангелина комп

In [202]:
def display_inf(inf):
    if type(inf) == dict:
        for key, value in dialog.items():
            print(f'{key}: {display_inf(value)}')
    else:
        print(inf)

In [204]:
for dlg, info in dialogs.items():
    display(info)
    print('-----------')

{'company_name': 'Диджитал бизнес',
 'manager_name': 'Ангелина',
 'greeting_message': {1: 'Алло здравствуйте'},
 'goodbye_message': {108: 'Всего хорошего до свидания'},
 'manager_self_presentation': {3: 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'},
 'requirement_done': True}

-----------


{'company_name': 'Диджитал бизнес',
 'manager_name': 'Ангелина',
 'greeting_message': {1: 'Алло здравствуйте'},
 'goodbye_message': {53: 'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
  54: 'До свидания'},
 'manager_self_presentation': {2: 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается'},
 'requirement_done': True}

-----------


{'company_name': 'Диджитал бизнес',
 'manager_name': 'Ангелина',
 'greeting_message': {2: 'Алло здравствуйте'},
 'goodbye_message': {},
 'manager_self_presentation': {3: 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там'},
 'requirement_done': False}

...

-----------


{'manager_name': 'unknown',
 'company_name': 'unknown',
 'greeting_message': {},
 'goodbye_message': {33: 'Во вторник все ну с вами да тогда до вторника до свидания'},
 'manager_self_presentation': {},
 'requirement_done': False}

-----------


{'manager_name': 'Анастасия',
 'company_name': 'unknown',
 'greeting_message': {},
 'goodbye_message': {142: 'Ну до свидания хорошего вечера'},
 'manager_self_presentation': {1: 'Да это анастасия'},
 'requirement_done': False}

-----------
